In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from importlib import import_module

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import yaml

from dist import models

In [ ]:
def run_one(name):
    gdf = gpd.read_file(f"dist/data/{name}.geojson")
    config = yaml.safe_load(open(f"dist/config/{name}.yml"))
    model = import_module(f"dist.models.{name}").entry
    pars = {p["col"]: p["val"] for p in config["pars"]}

    proc = gpd.GeoDataFrame(
        pd.concat([row, pd.Series(model(row.to_dict(), pars))]) for _, row in gdf.iterrows()
    )

    fig, ax = plt.subplots(figsize=(10, 10))
    var = "tech"
    proc.plot(ax=ax, column=var)
    ax.set_title(f"Model: {name} | Variable: {var}")
    plt.show()

In [ ]:
for n in ["fish", "irri", "hydro"]:
    run_one(n)